## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
%%time
import numpy as np
A = np.random.randint(0, 10000, 10**6)
print('A = ',A)
B = A+100
print('B = ',B)
print('Среднее значение - ', np.mean(B))

A =  [1508 2518  887 ... 2883 6818  512]
B =  [1608 2618  987 ... 2983 6918  612]
Среднее значение -  5104.758098
CPU times: total: 422 ms
Wall time: 540 ms


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [2]:
%%time
import numpy as np
vals = np.random.randint((2*10**6),size = (2*10**6,4))
alphabet = list(map(chr, range(ord('a'), ord('z')+1)))
alphabet += list(map(lambda x: x.upper(), alphabet))
first_5 = alphabet[:5]
first_5 +=list(map(lambda x: x.upper(), first_5))

a = np.random.choice(alphabet, 2*10**6)

a.shape = (2000000,1)
result = np.concatenate((a,vals ), axis=1)
res = result[np.isin(result[:, 0], first_5)]
print(res.shape[0])

384014
Wall time: 5.09 s


## Лабораторная работа 3

In [3]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [4]:
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [5]:
%%time
#A. С использованием метода DataFrame.iterrows исходной таблицы;
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1
count_rating1()

CPU times: total: 9.92 s
Wall time: 10 s


4.4544402182900615

In [6]:
%%time
#Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;
def count_rating2():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0 
    reviews1 = reviews[reviews['date'].dt.year==2010]
    for index, row in reviews1.iterrows():
        summ+=row['rating']

    answer2 = (summ/reviews1.shape[0])
    return answer2
count_rating2()

CPU times: total: 2.11 s
Wall time: 2.3 s


4.4544402182900615

In [7]:
%%time
#В. С использованием метода Series.mean
def count_rating3():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    answer3 = reviews[reviews['date'].dt.year==2010].rating.mean()
    return answer3
count_rating3()

CPU times: total: 1.16 s
Wall time: 1.2 s


4.4544402182900615

In [8]:
count_rating1()==count_rating2()==count_rating3()

True

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

### 1A - самая медленная функция



In [3]:
!pip install line_profiler

     ---------------------------------------- 83.6/83.6 kB 2.4 MB/s eta 0:00:00


In [4]:
!pip install memory_profiler


In [11]:
%load_ext line_profiler

In [12]:
%reload_ext line_profiler

In [13]:
%load_ext memory_profiler

In [14]:
%%writefile demo.py
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1


Overwriting demo.py


In [6]:
import cProfile, pstats, io


def profile(fnc):
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner
@profile
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1


In [7]:
count_rating1()

NameError: name 'pd' is not defined

In [17]:
#блоки программы, занимающие наибольшее время - считывание файла и итерации

In [18]:
%%time
#альтернативное решение 
def count_rating4():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    reviews1 = reviews[reviews['date'].dt.year==2010]
    summ_rating = reviews1.rating.sum()
    return summ_rating/reviews1.shape[0]
count_rating4()

CPU times: total: 1.19 s
Wall time: 1.21 s


4.4544402182900615

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [19]:
# @profile
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [20]:
%%time
get_word_reviews_count(reviews)

CPU times: total: 30.6 s
Wall time: 30.8 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

Узкие места кода - итерирование, создание новых элементов

In [21]:
reviews

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [22]:
reviews.iloc[0].review.lower()

"last week whole sides of frozen salmon fillet was on sale in my local supermarket, so i bought tons (okay, only 3, but total weight was over 10 pounds).  this recipe is perfect for salmon fillet, even though it calls for salmon steaks.  i cut up the salmon into individual portions and followed the instructions exactly.  i'm on one of those food combining diets, so i left out the white wine but added just a dash of white wine vinegar instead (just a little bit, not enough to change the taste of the dish).  super yummy, and leftovers for lunch today (lucky me)!"

In [23]:
%%time
def get_word_reviews_count_2(df):
    df['review'] = df['review'].fillna("")
    df['review'] = df['review'].apply(lambda x: ' '.join(set(x.split(' '))))
    return (pd.Series(' '.join(df.review).split()).value_counts())

get_word_reviews_count_2(reviews)

CPU times: total: 5.09 s
Wall time: 5.19 s


I                     101778
and                    96186
the                    92824
a                      82412
for                    74702
                       ...  
(obviously)                1
bowlful!                   1
proportions/taste          1
GOOOOD!!!!...Sorry         1
44th                       1
Length: 164271, dtype: int64

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [24]:
reviews = reviews[reviews.rating !=0]
mean_rating = reviews.rating.mean()
mean_rating

4.6611588859881055

In [52]:
%%time
#A
def mape_A():
    n = reviews.shape[0]
    y = [abs(rating - mean_rating) for rating in reviews.rating]
    percentage_deviation = [(t / mean_rating) * 100 for t in y]
    mape = sum(percentage_deviation) / n
    return mape

mape_A()

CPU times: total: 62.5 ms
Wall time: 64.8 ms


11.079150232255099

In [8]:
import numba

In [9]:
%%time

@numba.jit(nopython=True)
def mape_B(reviews):
    n = len(reviews)
    mean_rating = sum(reviews) / n
    absolute_errors = []
    for rating in reviews:
        absolute_errors.append(abs(rating - mean_rating))
    percentage_errors = []
    for error in absolute_errors:
        percentage_errors.append(error / mean_rating)
    mape = sum(percentage_errors) * 100 / n
    return mape
mape_numba(list(reviews.rating.values))

NameError: name 'mape_numba' is not defined

In [57]:
%%time
import numpy as np
def mape_C():
    n = reviews.shape[0]
    mean_rating = np.mean(reviews.rating)
    absolute_errors = np.abs(reviews.rating - mean_rating)
    percentage_errors = absolute_errors / mean_rating
    mape = np.sum(percentage_errors) * 100 / n
    return mape
mape_B()

CPU times: total: 0 ns
Wall time: 14.2 ms


11.07915023226724

In [63]:
%%time
import numpy as np
import numba

@numba.jit(nopython=True)
def mape_D(ratings_array):
    n = len(ratings_array)
    mean_rating = np.mean(ratings_array)
    absolute_errors = np.abs(ratings_array - mean_rating)
    percentage_errors = absolute_errors / mean_rating
    mape = np.sum(percentage_errors) * 100 / n
    return mape
mape_C(reviews.rating.values)

CPU times: total: 250 ms
Wall time: 244 ms


11.079150232279916

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4